In [1]:
from jetbot import Camera, bgr8_to_jpeg, Robot
import ipywidgets.widgets as widgets
from IPython.display import display
import traitlets, glob, os, cv2, math, time
from uuid import uuid1
import numpy as np
from pathlib import Path

In [2]:
robot = Robot()
camera = Camera.instance()

# Configure drive function

In [3]:
controller = widgets.Controller(index=0)

controller_container = widgets.HBox([controller])

## Test Contoller

In [4]:
# define a steearing traitlet to observe and combine the button events

SPEED = 0.2

class Steering(traitlets.HasTraits):
    left_motor = traitlets.Float()
    right_motor = traitlets.Float()
    def __init__(self, _acc, _brake, _left_right):
        
        self.acc = _acc
        self.brake = _brake
        self.left_right = _left_right
        super().__init__()
        self.set_notifiers()
        
    def func(self, change):
        speed = -(self.acc.value * SPEED) if self.brake.value > 0 else self.acc.value * SPEED
        self.right_motor = speed if self.left_right.value < (-0.5) else (speed * 0.5)
        self.left_motor = speed if self.left_right.value > (0.5) else (speed * 0.5)
        # print(change)
        
    def set_notifiers(self):
        traitlets.HasTraits.observe(self.acc, self.func, 'value')
        traitlets.HasTraits.observe(self.brake, self.func, 'value')
        traitlets.HasTraits.observe(self.left_right, self.func, 'value')

stearing = Steering(controller.buttons[1], controller.buttons[2], controller.axes[0])

# Create Data Folder

In [2]:
dataset_dir = os.path.join(os.getcwd(), 'dataset')
traffic_dir = os.path.join(dataset_dir, 'traffic')
blocked_dir = os.path.join(dataset_dir, 'blocked')
free_dir    = os.path.join(dataset_dir, 'free')

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    #os.makedirs(dataset_dir)
    Path(dataset_dir).mkdir(parents=True, exist_ok=True)
    Path(traffic_dir).mkdir(parents=True, exist_ok=True)
    Path(blocked_dir).mkdir(parents=True, exist_ok=True)
    Path(free_dir).mkdir(parents=True, exist_ok=True)
except FileExistsError:
    print('Directories not created becasue they already exist')

# Functions  
Functions to capture x y, traffic lights (R,G,A), and blocked data

In [6]:
def display_xy(camera_image):
    '''
    Display Direction
    '''
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image    

def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, uuid1())

def get_xy_path(x, y, path, label):
    '''
    This functions is to generate x (left right), y (forward, back) data on file name
    label: R=Red Light, A=Yellow Light, G=Green Light, B = Blocked, F = Free
    '''
    # Calculate i based on number of files in the directories
    i = len(glob.glob(os.path.join(path, '*.jpg'))) +1
    
    ## Final file name format
    ## xy_XXX_YYY_<Label>_<Number_Of_Files>.jpg
    ## Ideally each class at least to have 100 images for good result. 
    ## (Traffic lights will be 100 each)
    
    image_path = os.path.join(path,'xy_%03d_%03d_%s_%d.jpg' % (x * 50 + 50, y * 50 + 50,label, i))
    
    return image_path

## Traffic Light

Capture traffic lights image then label with different tag on file name.  
* R for Red
* A for Amber
* G for Green  

This will required 3 buttons to be used. (L1, R1, R2)

In [7]:
def save_red_light(change):
    if change['new']:
        path  = traffic_dir
        label = 'R'
        image_path = get_xy_path(x_slider.value, y_slider.value, path, label)
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        traffic_count_widget.value = len(glob.glob(os.path.join(path, '*.jpg')))
        
def save_amber_light(change):
    if change['new']:
        path  = traffic_dir
        label = 'A'
        image_path = get_xy_path(x_slider.value, y_slider.value, path, label)
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        traffic_count_widget.value = len(glob.glob(os.path.join(path, '*.jpg')))
        
def save_green_light(change):
    if change['new']:
        path  = traffic_dir
        label = 'G'
        image_path = get_xy_path(x_slider.value, y_slider.value, path, label)
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        traffic_count_widget.value = len(glob.glob(os.path.join(path, '*.jpg')))

## Blocked Object

In [8]:
def save_blocked_snapshot(change):
    if change['new']:
        path  = blocked_dir
        label = 'B'
        image_path = get_xy_path(x_slider.value, y_slider.value, path, label)
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        blk_count_widget.value = len(glob.glob(os.path.join(path, '*.jpg')))

def save_free_snapshot(change):
    if change['new']:
        path  = free_dir
        label = 'F'
        image_path = get_xy_path(x_slider.value, y_slider.value, path, label)
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        fre_count_widget.value = len(glob.glob(os.path.join(path, '*.jpg')))



## Setup buttons

In [9]:
display(controller_container)

* L1: Red
* R1: Amber
* R2: Green
* Y : Blocked
* X : Free

In [10]:
## Free up all buttons. Execute this whenever the functions has changed.
for b in controller.buttons:
    b.unobserve_all()

controller.buttons[4].observe(save_red_light, names='value')
controller.buttons[5].observe(save_amber_light, names='value')
controller.buttons[7].observe(save_green_light, names='value')

controller.buttons[0].observe(save_blocked_snapshot, names='value')
controller.buttons[3].observe(save_free_snapshot, names='value')

## WIDGETS for Capture Data

In [16]:
image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)
traffic_count_widget = widgets.IntText(description='Traffic', value=len(glob.glob(os.path.join(traffic_dir, '*.jpg'))))
blk_count_widget = widgets.IntText(description='Block', value=len(glob.glob(os.path.join(blocked_dir, '*.jpg'))))
fre_count_widget = widgets.IntText(description='Free', value=len(glob.glob(os.path.join(free_dir, '*.jpg'))))
x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')
display(image_widget)

Image(value=b'', format='jpeg', height='224', width='224')

## LINK Up Widget, Robot, and Game Controller

In [17]:
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)     

widgets.jsdlink((controller.axes[0], 'value'), (x_slider, 'value'))
widgets.jsdlink((controller.axes[1], 'value'), (y_slider, 'value'))

try: left_link
except NameError: left_link = None
if left_link:
    left_link.unlink()
left_link = traitlets.dlink((stearing, 'left_motor'), (robot.left_motor, 'value'))

try: right_link
except NameError: right_link = None
if right_link:
    right_link.unlink()
right_link = traitlets.dlink((stearing, 'right_motor'), (robot.right_motor, 'value'))

In [18]:
#display(widgets.HBox([image_widget, target_widget]), x_slider, y_slider)
display(widgets.VBox([
    widgets.HBox([target_widget,widgets.VBox([traffic_count_widget,blk_count_widget,fre_count_widget])]),
    widgets.HBox([x_slider,y_slider])
]))

* L1: Red
* R1: Amber
* R2: Green
* Y : Blocked
* X : Free

# Zip 

In [14]:
robot.stop()

In [3]:
!zip -r -q dataset.zip {dataset_dir}